## Task 3

In [ ]:
# Imports and BigQuery setup
import pandas as pd
import pandas_gbq
import sqlite3
from google.cloud import bigquery
from google.oauth2 import service_account

# BigQuery authentication and connection setup
service_path = ""
service_file = 'umt-msba-gg-key.json'
gbq_proj_id ='umt-msba'
private_key = service_path + service_file

credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

In [ ]:
# Query 1: Sales by date by hour
query1 = """
SELECT
    DATE(datetime) AS formatted_date,
    EXTRACT(hour FROM TIMESTAMP(datetime)) AS hour,
    ROUND(SUM(total), 2) AS spend,
    COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime), register_no, emp_no, trans_no))) AS trans,
    SUM(CASE WHEN trans_status IN ('R', 'V') THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department NOT IN (0, 15) AND
      (trans_status IS NULL OR trans_status IN ('R', 'V', ' '))
GROUP BY formatted_date, hour
ORDER BY formatted_date, hour
"""
sales_date_hour = pandas_gbq.read_gbq(query1, project_id=gbq_proj_id, credentials=credentials)

In [ ]:
# Query 2: Sales by owner by year by month
query2 = """
SELECT card_no, 
       EXTRACT(YEAR FROM datetime) as year,
       EXTRACT(MONTH FROM datetime) as month,
       SUM(total) as tot_spend,
       COUNT(DISTINCT CONCAT(
           CAST(EXTRACT(DATE FROM datetime) AS STRING), 
           CAST(register_no AS STRING), 
           CAST(emp_no AS STRING), 
           CAST(trans_no AS STRING))) as no_trans, 
       SUM(CASE WHEN trans_status IN ('V','R') THEN -1 ELSE 1 END) AS no_items
FROM `umt-msba.transactions.transArchive_*`
WHERE department NOT IN (0, 15) AND
      (trans_status IS NULL OR trans_status IN (' ', 'V', 'R'))
GROUP BY card_no, year, month
ORDER BY year, month
"""
sales_by_owner_ym = pandas_gbq.read_gbq(query2, project_id=gbq_proj_id, credentials=credentials)

In [ ]:
# Query 3: Sales by product description by year by month
query3 ="""
SELECT transacts.upc,
       transacts.description AS prod_descript,
       transacts.department AS departments,
       deptlu.dept_name,
       EXTRACT(YEAR FROM transacts.datetime) AS year,
       EXTRACT(MONTH FROM transacts.datetime) AS month,
       SUM(transacts.total) AS tot_spend,
       COUNT(DISTINCT CONCAT(
           CAST(EXTRACT(DATE FROM transacts.datetime) AS STRING), 
           CAST(transacts.register_no AS STRING), 
           CAST(transacts.emp_no AS STRING), 
           CAST(transacts.trans_no AS STRING))) as no_trans, 
       SUM(CASE WHEN transacts.trans_status IN ('V','R') THEN -1 ELSE 1 END) AS no_items
FROM `umt-msba.transactions.transArchive_*` transacts
LEFT JOIN `umt-msba.transactions.department_lookup` deptlu 
ON transacts.department = deptlu.department
WHERE transacts.department NOT IN (0, 15)
      AND (transacts.trans_status IS NULL OR transacts.trans_status IN (' ', 'V', 'R'))
GROUP BY prod_descript, year, month, transacts.upc, departments, deptlu.dept_name
ORDER BY tot_spend DESC;
"""
sales_by_prod_ym = pandas_gbq.read_gbq(query3, project_id=gbq_proj_id, credentials=credentials)

In [ ]:
# Creating a connection to the SQLite database
db_conn = sqlite3.connect("wedge_report.db")

# Populating the database with the DataFrames
sales_date_hour.to_sql(name='sales_date_hour', con=db_conn, if_exists='replace', index=False)
sales_by_owner_ym.to_sql(name='sales_by_owner_ym', con=db_conn, if_exists='replace', index=False)
sales_by_prod_ym.to_sql(name='sales_by_prod_ym', con=db_conn, if_exists='replace', index=False)

# Closing the database connection
db_conn.close()